# Intel® Extension for Scikit-learn Random Forest for Yolanda dataset

In [2]:
from timeit import default_timer as timer
from sklearn import metrics
from IPython.display import HTML
from sklearn.datasets import fetch_openml
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from DataScience.connection import *
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.ensemble import RandomForestRegressor
import joblib
import os
import logging
import pandas as pd
import datetime

In [1]:
# x, y = fetch_openml(name="Yolanda", return_X_y=True)
x, y = make_regression(n_samples=1000000, n_features=100, n_informative=10, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=72)
x_train.shape, x_test.shape, y_train.shape, y_test.shape
# Shrink for experimental use case
x_train = x_train[:1000]
y_train= y_train[:1000]

session = create_snowflake_session()
session.file.get(
    stage_location=os.path.join("@CONTAINER_DEMO_DB.PUBLIC.RDF_MODEL", "yolanda.joblib"),
    target_directory='.'
)
model = joblib.load('yolanda.joblib')
os.unlink('yolanda.joblib')
display(model)
preds = model.predict(x_test[:10])

# For Demo Purpose
df = pd.DataFrame(preds, columns=['predictions'])
df['systime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
_ = session.write_pandas(df = df, table_name='PREDICTION_EXAMPLE', auto_create_table=True, overwrite=False)
session.sql("SELECT * FROM PREDICTION_EXAMPLE LIMIT 10").show()

In [ ]:
session.sql("SELECT COUNT(1) FROM PREDICTION_EXAMPLE").show()